In [ ]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer


## Visualisation des données originales

In [38]:
df = pd.read_csv("/home/antoine/mlops_reco_movies/airflow/data/raw/bronze/movies.csv")

df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [39]:
df.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [40]:
# Renommer columns & typage
df = df.astype({"movieId": "int32"}).rename(columns={"movieId": "movie_id"})
df.dtypes

movie_id     int32
title       object
genres      object
dtype: object

In [41]:
# Extraction de l'année
df["year"] = (
            df["title"]
            .str.extract(r"(?:\(|\[)?(\d{4})(?:\)|\]| TV)?")[0]
            .astype(float)
            .astype("Int64")
        )

df.head()

,movie_id,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [42]:
df['year'].describe()

count        27259.0
mean     1990.851315
std       110.971987
min           1000.0
25%           1976.0
50%           1998.0
75%           2008.0
max           9500.0
Name: year, dtype: Float64

On observe que les dates sont globalement bonnes mais nous avons des outliers. Les premiers films ayant été créé en 1888, on va définir cette année comme année minimum et en année maximum celle d'aujourdh'hui. 
On remplace les outliers par des Nan, pour ensuite applique une valeur médiane à ces éléments, ainsi qu'au Nan d'origines. 

In [43]:
ANNEE_MIN = 1888
ANNEE_MAX = 2025

# Remplacer les années aberrantes par NaN
df.loc[(df["year"] < ANNEE_MIN) | (df["year"] > ANNEE_MAX), "year"] = np.nan

# Imputation par la médiane des valeurs valides
median_year = df.loc[df["year"].between(ANNEE_MIN, ANNEE_MAX), "year"].median()
df["year"] = df["year"].fillna(median_year)

df['year'].describe()

count        27278.0
mean     1989.350722
std        23.352755
min           1891.0
25%           1976.0
50%           1998.0
75%           2008.0
max           2019.0
Name: year, dtype: Float64

In [44]:
print(df.year.unique())

<IntegerArray>
[1995, 1994, 1996, 1976, 1992, 1988, 1967, 1993, 1964, 1977,
 ...
 1893, 2013, 1896, 2016, 1911, 2014, 1895, 2015, 1905, 1891]
Length: 121, dtype: Int64


In [45]:
# Nettoyage du titre
df["clean_title"] = df["title"].str.replace(r"\s*[\[(]\d{4}[\])]\s*", "", regex=True)

df.head()

,movie_id,title,genres,year,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II


In [46]:
# Gestion des genres avec seuillage
df["genres"] = df["genres"].str.replace("(no genres listed)|^$", "Unknown", regex=True)
genre_lists = df["genres"].str.split("|")

In [47]:
print(genre_lists.head())

0    [Adventure, Animation, Children, Comedy, Fantasy]
1                       [Adventure, Children, Fantasy]
2                                    [Comedy, Romance]
3                             [Comedy, Drama, Romance]
4                                             [Comedy]
Name: genres, dtype: object


In [48]:
df["genres_list"] = genre_lists
df["genres"] = genre_lists.str.join(", ")
df.head()

,movie_id,title,genres,year,clean_title,genres_list
0,1,Toy Story (1995),"Adventure, Animation, Children, Comedy, Fantasy",1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"Adventure, Children, Fantasy",1995,Jumanji,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"Comedy, Romance",1995,Grumpier Old Men,"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"Comedy, Drama, Romance",1995,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II,[Comedy]


In [49]:
mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(genre_lists), columns=mlb.classes_, index=df.index)
df = pd.concat([df, genres_encoded], axis=1)
df.head()

,movie_id,title,genres,year,clean_title,genres_list,(Unknown),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),"Adventure, Animation, Children, Comedy, Fantasy",1995,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"Adventure, Children, Fantasy",1995,Jumanji,"[Adventure, Children, Fantasy]",0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"Comedy, Romance",1995,Grumpier Old Men,"[Comedy, Romance]",0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),"Comedy, Drama, Romance",1995,Waiting to Exhale,"[Comedy, Drama, Romance]",0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,1995,Father of the Bride Part II,[Comedy],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
print(df.columns)

Index(['movie_id', 'title', 'genres', 'year', 'clean_title', 'genres_list',
       '(Unknown)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')
